In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

import models
import confusion
import config

%load_ext line_profiler

Using TensorFlow backend.


In [2]:
tf_version = tf.__version__
tf_version

'1.10.0'

In [3]:
# speak softly
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)

### Model Selection

#### TPU Accelerator on Raspberry Pi

#### GPU or Standard TensorFlow

In [4]:
m = models.ClassifyRegular()

### Profile Data Name  
Label for statistics compilation  

In [5]:
profile_name = "win10_i7_tf_{}CPU_A".format(tf_version)
profile_name

'win10_i7_tf1.10.0CPU_A'

### Load MobileNetV2

In [6]:
m.load_model()

### Test on single known image of a German Shepherd

In [7]:
m.predict_file(config.download_directory + 'Female_German_Shepherd.jpg')

'german shepherd'

### Run Classification on all Stanford Dog Images

In [ ]:
p = %prun -r m.predict_dataset(config.download_directory + \
                               os.path.sep + 'Images', verbose=False)

### Save Profile Results

In [ ]:
p.dump_stats(config.data_directory + os.path.sep + 
             profile_name + "_pstats.txt")

### Collate Predictions and Evaluate Model Performance

In [ ]:
m.collate_predictions()

In [ ]:
df = m.df

In [ ]:
df.head()

In [ ]:
dog_names = df.y_true.unique()
predicted_labels = df.y_pred.unique()

#### Clean Results  
Assign the predicted label to 'other' if it is not a dog name

In [ ]:
df.y_pred = df.y_pred.apply(lambda x: x if x in dog_names else 'other')

Insert another label to hold the 'other' labels

In [ ]:
df = df.append(pd.DataFrame({'y_true':'other','y_pred':'other'}, index=[0]),
                ignore_index=True)

##### Overall classification accuracy

In [ ]:
acc = df.y_pred == df.y_true
acc.sum()/len(acc)

##### Classification accuracy of labeling it some breed of dog

In [ ]:
df['y_pred_dog'] = df.y_pred.apply(lambda x: True if x in dog_names else False)
df.y_pred_dog.sum()/len(df)

In [ ]:
df.head()

Save predictions for comparison

In [ ]:
df.to_csv(config.data_directory + os.path.sep + 
          profile_name+'_predictions.csv', sep=',', index=False)

### Confusion Matrix Plot

In [ ]:
cm = confusion.Matrix(y=df.y_true, p=df.y_pred)

In [ ]:
cm.a

In [ ]:
fig = cm.plot(cmap='Greens', font_scale=0.75, 
        annot_kws={"size": 14}, ticklabels=True, 
        annot=False, figsize=30)